In [ ]:
import arcgis
import os
import pandas as pd
import numpy as np

In [ ]:
with open(f"..{os.sep}sample_data.json", "r") as f:
    df = pd.read_json(f, orient="records", date_unit="ms")
df = arcgis.features.SpatialDataFrame(df)

In [ ]:
def dissolve_sdf(df, by=None):
    def dissolve_shapes(row, sr):
        tmp = arcgis.gis.GIS()
        geom = arcgis.geometry.functions.union(geometries=row.SHAPE, spatial_ref=sr, gis=tmp)
        return geom
    
    if by:
        na = df.fillna({f: "NONE" for f in by})
        grouped = na.groupby(by)
        listed_geoms = grouped.SHAPE.apply(list)
        listed_geoms = listed_geoms.reset_index()
        dissolved = listed_geoms.replace("NONE", np.NaN)
    else:
        geoms = [df.SHAPE.to_list()]
        dissolved = pd.DataFrame([geoms], columns=["SHAPE"])

    dissolved.SHAPE = dissolved.apply(dissolve_shapes, sr=2876, axis=1)

    return dissolved

In [ ]:
fields = ["FLOODPLAIN", "FEMAZONE", "DRAINAGE", "LIFECYCLE", "ADOPTDATE", "INEFFDATE", "SOURCE"]
diss = dissolve_sdf(df, fields)
diss